# Khám phá dữ liệu

Tạm thời dừng việc làm sạch dữ liệu, ta chuyển sang xem có thể rút ra được những nhận xét gì từ bộ data này.

Trước hết cứ load lại dữ liệu

In [ ]:
import pandas as pd

from loaddata import data

In [ ]:
data.info()

Ngoài `info()` thì còn có hàm `describe()` mô tả cụ thể hơn

In [ ]:
data.describe()

Mặc định thì `describe()` chỉ mô tả các cột có kiểu dữ liệu số. Nếu muốn hiện hết thì có thể dùng parameter `include`

In [ ]:
data.describe(include='all')

Cũng có thể dùng `include` để chọn ra những loại dữ liệu cụ thể

In [ ]:
import numpy as np

data.describe(include=[np.number, 'category'])

### Exercise

Dùng `describe()` để mô tả riêng cột `Embarked`

In [ ]:
# Code vào đây

In [ ]:
%load solutions/describe_embarked.py

## Nhận biết các feature quan trọng

Câu hỏi quan trọng nhất trong bài toán này là "những yếu tố nào ảnh hưởng đến khả năng sống còn của 1 người trên tàu Titanic?". Để trả lời, ta có thể xem xét mối quan hệ giữa từng cột với cột `Survived`. Tuy nhiên ta cứ thử đưa ra 1 vài dự đoán trước:
- `Pclass`? Người có hạng vé cao có thể có ưu tiên áo phao
- `Sex`? Phụ nữ được ưu tiên, cái này khá chắc chắn
- `Age`? Mấy anh trẻ khỏe đương nhiên dễ sống hơn mấy ông già rồi
- `Name`? Có thể thấy tên bao gồm cả _title_, ví dụ `Capt`, `Lady`... thể hiện địa vị xã hội của người đó. Ngoài ra có thể lấy ra họ của từng người, từ đó đoán thêm 1 số thông tin như sắc tộc, xuất xứ...
- Số lượng anh em, vợ/chồng `SibSp`? Bố mẹ con cái `Parch`? Có thể vì cứu nhau mà cùng sống hoặc cùng chết.
- Giá vé `Fare`? Nếu giá vé quyết định khoang ở thì có thể có ảnh hưởng, vì có khoang chìm trước khoang chìm sau.
- Điểm lên tàu `Embarked`? Tương tự `Fare`, có thể nó có ảnh hưởng đến khoang ở trên tàu.

Hầu hết các cột đã ở dạng rất dễ tham chiếu, chỉ có cột `Name` là có nhiều data thừa. Tạm không đụng đến họ mà chỉ quan tâm đến title, ta nên bóc tách phần thông tin cần thiết ra để dễ tham chiếu sau này.

Ngoài ra, có thể tính được số người trong gia đình trên tàu bằng cách cộng `SibSp` và `Parch`

## Feature engineering

Trước hết ta cần 1 function để lấy được title từ tên người. Regular Expression là perfect cho việc này. Thư viện Regular Expression trong Python là `re`

In [ ]:
import re

def get_title_from_name(name):
    m = re.search(' ([A-Za-z]*)\.', name)
    if not m:
        raise ValueError(f'Name doesn\'t match pattern: {name}')
    return m.group(1)

# Test qua
for i in range(1, 10):
    print(get_title_from_name(data.loc[i, 'Name']))

Trông có vẻ ổn rồi. Giờ áp dụng trên toàn bộ cột `Name` để tạo ra cột mới `Title`

In [ ]:
data['Title'] = data['Name'].apply(get_title_from_name).astype('category')
data['Title']

Cột `Name` đã không còn giá trị, xóa

In [ ]:
data.drop(columns='Name', inplace=True)

Thử xem tần suất xuất hiện của từng title như thế nào

In [ ]:
with pd.option_context('max.rows', 20):
    print(data['Title'].value_counts())

Chú ý:
- `Mlle` (tiếng Pháp) = `Miss` (tiếng Anh), `Mme` (tiếng Pháp) = `Mrs` (tiếng Anh)
- Chỉ có 1 `Ms`, không đáng để để thành 1 category riêng, có thể ghép vào với `Miss`
- Ngoài `Mr`, `Miss`, `Mrs`, `Mlle`, `Mme`, `Ms`, `Master` thì các title còn lại đều dành cho quý tộc/người được trọng vọng, và có tần suất xuất hiện rất thấp. Có thể gộp vào thành 1 category `High standing`

In [ ]:
# Đổi Mlle, Ms -> Miss, Mme -> Mrs
data['Title'] = data['Title'].replace(['Mlle', 'Ms'], 'Miss')
data['Title'] = data['Title'].replace(['Mme'], 'Mrs')

with pd.option_context('max.rows', 20):
    print(data['Title'].value_counts())

Các category còn lại thì lằng nhằng hơn do pandas không có hàm merge category. Có thể xóa toàn bộ các category này để biến các value tương ứng bị `null`, rồi điền `High standing` vào tất cả các chỗ `null`

In [ ]:
data['Title'].cat.set_categories(('Mr', 'Mrs', 'Miss', 'Master', 'High Standing'), inplace=True)
data['Title'].fillna('High Standing', inplace=True)
data['Title'].value_counts()

### Exercise

Tạo thêm cột `FamilySize` sử dụng cột `SibSp` và `Parch`. Sau đó xóa bỏ 2 cột này.

In [ ]:
# Code vào đây

In [ ]:
# %load solutions/create_family_size.py
data['FamilySize'] = data['SibSp'] + data['Parch'] + 1  # Cộng thêm cả bản thân người đó nữa
data.drop(columns=['SibSp', 'Parch'], inplace=True)

## Visualize

Đã đến lúc xem xem những dự đoán ban đầu của chúng ta chính xác hay không. Ta sẽ so sánh từng cột với cột `Survived`

Để visualize data trong Python, có 2 thư viện phổ biến nhất là `matplotlib` và `seaborn`. Demo về các loại chart có thể xem ở [đây](https://matplotlib.org/gallery.html) và [đây](https://python-graph-gallery.com/).

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

Trước hết hãy thử xem tỉ lệ sống/chết như thế nào

In [ ]:
f, ax=plt.subplots(1, 2, figsize=(18, 8))
data['Survived'].value_counts().plot.pie(explode=[0, 0.1], autopct='%1.1f%%', ax=ax[0],shadow=True)
ax[0].set_title('Survived')
ax[0].set_ylabel('')
sns.countplot('Survived', data=data, ax=ax[1])
ax[1].set_title('Survived')
plt.show()

OK, 61.6% chết và 38.4% sống. Giờ hãy xem cụ thể từng cột

### Pclass

In [ ]:
pd.crosstab(data['Pclass'], data['Survived'], margins=True).style.background_gradient(cmap='summer_r')

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(18, 8))
data[['Pclass','Survived']].groupby(['Pclass']).mean().plot.bar(ax=ax[0])
ax[0].set_title('Survival rate by Pclass')
sns.countplot('Pclass', hue='Survived', data=data, ax=ax[1])
ax[1].set_title('Pclass: Survived vs Dead')
plt.show()

Có thể thấy mấy ông class 1 có tỉ lệ sống cao hơn chết, còn class 3 thì T__T. Có tiền có khác!

### Sex

In [ ]:
pd.crosstab(data['Sex'], data['Survived'], margins=True).style.background_gradient(cmap='summer_r')

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(18, 8))
data[['Sex','Survived']].groupby(['Sex']).mean().plot.bar(ax=ax[0])
ax[0].set_title('Survival rate by Sex')
sns.countplot('Sex', hue='Survived', data=data, ax=ax[1])
ax[1].set_title('Sex: Survived vs Dead')
plt.show()

Đúng là nữ giới được ưu tiên hơn. Cỡ 75% nữ sống trong khi nam chỉ được tầm 20%.

### Title

In [ ]:
pd.crosstab(data['Title'], data['Survived'], margins=True).style.background_gradient(cmap='summer_r')

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(18, 8))
data[['Title','Survived']].groupby(['Title']).mean().plot.bar(ax=ax[0])
ax[0].set_title('Survival rate by Title')
sns.countplot('Title', hue='Survived', data=data, ax=ax[1])
ax[1].set_title('Title: Survived vs Dead')
plt.show()

`Mrs` và `Miss` có tỉ lệ sống cao nhất, và thấp nhất là `Mr`. Cái này đơn giản là do giới tính. `Master` cũng có tỉ lệ sống rất cao, tức là trẻ con được ưu tiên. Mấy ông `High Standing` có tỉ lệ sống sót xấp xỉ tỉ lệ chung (38.4%), chứng tỏ có địa vị cao cũng không có nhiều giá trị lắm.

### Age

In [ ]:
msurv = data[(data['Survived'] == 1) & (data['Sex'] == 'male')]
fsurv = data[(data['Survived'] == 1) & (data['Sex'] == 'female')]
mnosurv = data[(data['Survived'] == 0) & (data['Sex'] == 'male')]
fnosurv = data[(data['Survived'] == 0) & (data['Sex'] == 'female')]

plt.figure(figsize=[13, 5])
plt.subplot(121)
sns.distplot(fsurv['Age'].values, bins=range(0, 85, 5), kde=False, color='blue')
sns.distplot(fnosurv['Age'].values, bins=range(0, 85, 5), kde=False, color='red', axlabel='Female Age')
plt.subplot(122)
sns.distplot(msurv['Age'].values, bins=range(0, 85, 5), kde=False, color='blue')
sns.distplot(mnosurv['Age'].values, bins=range(0, 85, 5), kde=False, color='red', axlabel='Male Age')
plt.show()

Trẻ con < 5 tuổi có tỉ lệ sống cao. 1 ông già 80 tuổi cũng sống sót. Tầm 15-25 tuổi chết rất nhiều. Phần lớn hành khách có độ tuổi 25-30.

### FamilySize

In [ ]:
pd.crosstab(data['FamilySize'], data['Survived'], margins=True).style.background_gradient(cmap='summer_r')

In [ ]:
sns.factorplot('FamilySize', 'Survived', data=data)
plt.show()

Đi 1 mình có tỉ lệ sống sót thấp, chắc không đánh lại các gia đình để giành thuyền/phao. Đi đông quá (> 4) cũng chết, theo đúng dự đoán ban đầu. Có điểm đáng chú ý là những gia đình lớn 8, 11 người hoàn toàn không ai sống sót. Lý do chắc là `Pclass`

In [ ]:
pd.crosstab(data['FamilySize'], data['Pclass'], margins=True).style.background_gradient(cmap='summer_r')

Đúng là `Pclass`. Các gia đình lớn đều thuộc `Pclass = 3`.

### Fare

In [ ]:
sns.lmplot(x='Age', y='Fare', data=data, hue='Survived', palette='Set1', fit_reg=False, size=7)
plt.show()

Giá vé từ khoảng $50 trở lên có vẻ sống nhiều hơn chết. Lý do chắc lại là do những người mua vé đắt tiền thường là thuộc `Pclass = 1` hoặc `2`

In [ ]:
f, ax = plt.subplots(1, 3, figsize=(20, 8))
for i in range(3):
    sns.distplot(data[data['Pclass'] == str(i + 1)]['Fare'], ax=ax[i])
    ax[i].set_title('Fares in Pclass %d' % (i + 1))

plt.show()

Trong nhóm `Pclass = 1` giá vé khá dao động, nhưng tập trung ở khoảng \$20-\$60. Nhóm `Pclass = 2` thì tập trung ở \$8-\$12 và \$24-\$28. Nhóm `Pclass = 3` hầu như chỉ mua vé dưới $10.

### Embarked

In [ ]:
sns.factorplot('Embarked', 'Survived', data=data)
plt.show()

Xuất phát từ cảng `C` có vẻ may mắn hơn cảng `S`.

### Exercise

Thử so sánh `Embarked` với các cột khác xem có thể biết được nguyên nhân xuất phát từ `C` lại có tỉ lệ sống cao nhất không

In [ ]:
# Code vào đây

In [ ]:
%load solutions/visualize.py